In [1]:
import pandas as pd
import pandera as pa


In [2]:
fp = "../data/online_retail_II.csv"
df = pd.read_csv(fp)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_5807/2586147146.py:2: DtypeWarning: Columns (3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.1,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067372 entries, 0 to 1067371
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   Invoice      1067372 non-null  object
 1   StockCode    1067372 non-null  object
 2   Description  1062990 non-null  object
 3   Quantity     1067372 non-null  object
 4   InvoiceDate  1067372 non-null  object
 5   Price        1067372 non-null  object
 6   Customer ID  824365 non-null   object
 7   Country      1067372 non-null  object
dtypes: object(8)
memory usage: 65.1+ MB


In [5]:
pa.infer_schema(df)

<Schema DataFrameSchema(columns={'Invoice': <Schema Column(name=Invoice, type=DataType(object))>, 'StockCode': <Schema Column(name=StockCode, type=DataType(object))>, 'Description': <Schema Column(name=Description, type=DataType(object))>, 'Quantity': <Schema Column(name=Quantity, type=DataType(object))>, 'InvoiceDate': <Schema Column(name=InvoiceDate, type=DataType(object))>, 'Price': <Schema Column(name=Price, type=DataType(object))>, 'Customer ID': <Schema Column(name=Customer ID, type=DataType(object))>, 'Country': <Schema Column(name=Country, type=DataType(object))>}, checks=[], index=<Schema Index(name=None, type=DataType(int64))>, coerce=True, dtype=None, strict=False, name=None, ordered=False, unique_column_names=False)>

In [6]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [7]:
# from pandera import Check, Column, DataFrameSchema
# from pandera.engines import pandas_engine


# retail_data_prelim_schema = DataFrameSchema({
#     "Invoice": Column(str, nullable=False),
#     #"StockCode":  Column(str, Check.str_matches(r'^[0-9A-Z]{5,}$'), nullable=False),
#     "StockCode":  Column(str, nullable=False),
#     "Description":  Column(str, nullable=False),
#     "Quantity": Column(int, nullable=False),
#     "InvoiceDate":  Column(pd.DatetimeTZDtype(unit="ns", tz="UTC"), nullable=False),
#     "Price": Column(float, Check.greater_than(0), nullable=False),
#     "Customer ID": Column(str, nullable=False),
#     "Country": Column(str, nullable=False)
# })

# retail_data_prelim_schema.validate(df)

In [8]:
from pandera.errors import SchemaErrors
invoice_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Invoice")
try:
    validated_invoices = invoice_schema.validate(df["Invoice"])
    print("Invoice data has values that are as expected")
except SchemaErrors as se:
    print("Invoice data needs analysis - has values that are unexpected!")
    print("Failure Cases:")
    print(se.failure_cases)

Invoice data has values that are as expected


In [9]:
stock_code_schema = pa.SeriesSchema(
    str,
    [pa.Check.str_matches(r'^[0-9A-Za-z]{4,}$')],
    coerce=True,
    nullable=False,
    name="StockCode")

In [10]:
failure_cases = None
try:
    validated_stock_code = stock_code_schema.validate(df["StockCode"])
    print("StockCode data has values that are as expected")
except pa.errors.SchemaErrors as err:
    print("StockCode data needs analysis - has values that are unexpected!")
    failure_cases = err.failure_cases
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)
    

SchemaError:          index failure_case
0         735            D
1         736            D
2        2379          DOT
3        2539          DOT
4        2551          DOT
...       ...          ...
3640  1063783            M
3641  1064830          DOT
3642  1066370          DOT
3643  1067002          DOT
3644  1067003            M

[3645 rows x 2 columns]


In [11]:
description_code_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Description")

In [12]:
failure_cases = None
try:
    description_code_validation = description_code_schema.validate(df["Description"])
    print("Description data has values that are as expected")
# except pa.errors.SchemaErrors as err:
#     print("Description data needs analysis - has values that are unexpected!")
#     failure_cases = err.failure_cases
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:          index failure_case
0         470          NaN
1        3114          NaN
2        3161          NaN
3        3731          NaN
4        4296          NaN
...       ...          ...
4377  1060784          NaN
4378  1060788          NaN
4379  1060794          NaN
4380  1062443          NaN
4381  1064016          NaN

[4382 rows x 2 columns]


In [13]:
quantity_schema = pa.SeriesSchema(
    int,
    [pa.Check.greater_than(0)],
    coerce=True,
    nullable=False,
    name="Quantity")

In [14]:
failure_cases = None
try:
    quantity_validation = quantity_schema.validate(df["Quantity"])
    print("Quantity data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:      index failure_case
0  525461     Quantity


In [15]:
df["Quantity"][525461]

'Quantity'

In [16]:
invoice_date_schema = pa.SeriesSchema(
    pd.DatetimeTZDtype(unit="ns", tz="UTC"),
    coerce=True,
    nullable=False,
    name="InvoiceDate")

In [17]:
failure_cases = None
try:
    invoice_date_validation = invoice_date_schema.validate(df["InvoiceDate"])
    print("InvoiceDate data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:      index failure_case
0  525461  InvoiceDate


In [18]:
df["InvoiceDate"][525461]

'InvoiceDate'

In [19]:
df = df.drop(index=525461)


In [20]:
df = df.reset_index(drop=True)

In [21]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.1,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.1,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/11 12:50,4.95,12680.0,France


In [22]:
failure_cases = None
try:
    quantity_validation = quantity_schema.validate(df["Quantity"])
    print("Quantity data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:           index  failure_case
0          178           -12
1          179            -6
2          180            -4
3          181            -6
4          182           -12
...        ...           ...
22945  1065910           -11
22946  1067002            -1
22947  1067176            -5
22948  1067177            -1
22949  1067178            -5

[22950 rows x 2 columns]


In [23]:
failure_cases = None
try:
    invoice_date_validation = invoice_date_schema.validate(df["InvoiceDate"])
    print("InvoiceDate data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

InvoiceDate data has values that are as expected


/Users/rajivsambasivan/opt/anaconda3/envs/rrenv/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:733: FutureWarning: Using .astype to convert from timezone-naive dtype to timezone-aware dtype is deprecated and will raise in a future version.  Use ser.dt.tz_localize instead.
  return col.astype(pandas_dtype)


In [24]:
price_schema = pa.SeriesSchema(
    float,
    [pa.Check.greater_than(0)],
    coerce=True,
    nullable=False,
    name="Price")

In [25]:
failure_cases = None
try:
    price_validation = price_schema.validate(df["Price"])
    print("Price data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:          index  failure_case
0         263           0.0
1         283           0.0
2         284           0.0
3         470           0.0
4        3114           0.0
...       ...           ...
6202  1062442           0.0
6203  1063965           0.0
6204  1063966           0.0
6205  1064015           0.0
6206  1064386           0.0

[6207 rows x 2 columns]


In [26]:
customer_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Customer ID")

In [27]:
failure_cases = None
try:
    customer_validation = customer_schema.validate(df["Customer ID"])
    print("Customer ID data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:            index failure_case
0           263          NaN
1           283          NaN
2           284          NaN
3           470          NaN
4           577          NaN
...         ...          ...
243002  1066997          NaN
243003  1066998          NaN
243004  1066999          NaN
243005  1067000          NaN
243006  1067001          NaN

[243007 rows x 2 columns]


In [28]:
country_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Country")

In [29]:
failure_cases = None
try:
    country_validation = country_schema.validate(df["Country"])
    print("Countrydata has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

Countrydata has values that are as expected
